Fecha: 20/09/2023

Realizado por: Mario Earles Otero Andrade

Materia: Machine Learning II

# Punto 1

1. In your own words, describe what vector embeddings are and what they are useful for.

Vector embeddings are numerical vector representations of objects, such as words or phrases, derived through machine learning models. They capture semantic relationships and are useful in various natural language processing tasks including similarity estimation, clustering, and classification. They effectively facilitate the mathematical manipulation of linguistic concepts, allowing computers to 'understand' textual data in a more nuanced manner.

# Punto 2

2. What do you think is the best distance criterion to estimate how far two embeddings (vectors) are from each other? Why?


The most used and effective criterion tends to be cosine similarity to determine how close two embeddings are to each other, as it focuses on the angle between two vectors, disregarding their magnitude. This is especially useful in high-dimensional spaces, common in natural language processing tasks, as it can capture semantic similarity more precisely than other distance metrics. However, the "best" metric can depend on the specific context and the characteristics of the data.

# Punto 3

3. Let us build a Q&A (question answering) system!😀For this, consider the following steps:

a. Pick whatever text you like, in the order of 20+ paragraphs.

b. Split that text into meaningful chunks/pieces.

c. Implement the embedding generation logic. Which tools and approaches would help you generate them easily and high-level?

d. For every question asked by the user, return a sorted list of the N chunks/pieces in your text that relate the most to the question. Do results make sense?

The exercise using a Hugging Face model, specifically using distilbert-base-multilingual-cased, which works well with multiple languages, including English.


In [2]:
#Install the necessary libraries
!pip install transformers torch

In [10]:
#Import libraries and prepare the text
import torch
import re
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')


In [11]:
#Functions to split the text into chunks
def split_into_paragraphs(text):
    return [p for p in re.split(r'\n+', text) if p]

def split_into_n_words(text, n=50):
    words = text.split()
    chunks = [' '.join(words[i:i+n]) for i in range(0, len(words), n)]
    return chunks

In [12]:
#Convert the text into embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        output = model(**inputs)
    return output.last_hidden_state.mean(dim=1)

In [13]:
#Given a question, we compute its embedding and then compute the cosine similarity with each paragraph's embedding
def get_relevant_chunks(question, embeddings, chunks):
    question_embedding = get_embedding(question)
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    scores = [(cos(question_embedding, emb), chunk) for emb, chunk in zip(embeddings, chunks)]
    scores.sort(key=lambda x: x[0], reverse=True)
    return [entry[1] for entry in scores]

In [23]:
# Your complete text
text_input = """
Los términos Analytics y Big Data son cada vez más socorridos y demandados en el entorno profesional a pesar de ser pocos los que verdaderamente conocen los principios que los sustentan. En este artículo nos adentramos en lo que se ha dado en denominar Big Data en Ingles, o lo que podemos traducir al español como la ciencia que permite extraer valor de grandes cantidades de datos. Algo que está revolucionando la forma en que se toman las decisiones que nos afectan en nuestro día a día. A continuación, detallaremos aspectos históricos, éticos y tecnológicos; analizando bondades y riesgos que acompañaremos con casos prácticos reales.
¿Qué es la Ciencia de Datos? Se trata de una disciplina que permite tomar decisiones informadas y soportadas en evidencias científicas combinando: Experiencia de negocio, para detectar necesidades y definir casos de uso; Conocimientos matemáticos, para dar respuestas fundamentadas; e Ingeniería Informática, para escalar los cálculos y realizarlos de forma optimizada.
¿Cómo es el proceso analítico? Como hemos dicho, todo empieza por tener datos primando siempre calidad frente al volumen. Una vez que tenemos esos datos, gracias a la matemática y la estadística, podemos analizar tendencias y relaciones entre los mismos para extrapolar conclusiones. En términos sencillos, buscar patrones que se repiten entre ellos.
Una vez hecho este análisis, seremos capaces, mediante otras técnicas, de algo que solo los ordenadores nos permiten hoy en día, crear modelos. Un modelo es una representación / simulación de la realidad en la que entran unos datos y salen otros. Estos modelos pueden ser de diversa índole, pero en términos generales hablamos de: predicciones, clasificaciones o recomendaciones.
Por hacerlo más claro, ya desde hace años un experto en una materia mediante observación y registro sencillo de datos, muchas veces en papel, era capaz de acertar con una precisión impecable una determinada avería. Un ejemplo claro lo tenéis en un mecánico de un coche. Esto es gracias a que el cerebro humano también analiza lo que ve y busca patrones y es capaz de inferir relaciones de correlación y/o causalidad. Note el lector más atento que correlación no implica causalidad.
Lo que la Ciencia de Datos nos ha brindado es hacerlo: de una forma más precisa, manejando muchos más datos, evaluando muchos más patrones sin olvidar nada y no siendo dependiente del experto.
Una vez cargado el modelo en un ordenador, podemos automatizar el proceso de análisis y la obtención de resultados. Todo ello para cualquier proceso que podamos imaginar. Es posible dar otro paso más, tratando de acercar la inteligencia artificial al modo en que los humanos tomamos decisiones.
Dicho de otra forma, este modelo puede evolucionar a medida que trabaja, lee datos, produce una salida y se alimenta de los resultados reales auto ajustándose (aprendiendo) sobre la marcha. Esto es parte de lo que se denomina aprendizaje de las maquinas o 'Machine Learning' y su desarrollo y generalización ha sido posible gracias a la disponibilidad de grandes cantidades de datos, el abaratamiento de los costes, el incremento de la capacidad de procesado y la generalización del acceso a las técnicas y algoritmos
¿Cómo ha evolucionado la Ciencia de Datos? Tras la II Guerra Mundial el taylorismo se adopta de manera generalizada a la par que los ordenadores se investigan extensamente. En 1962 John W. Tukey estadístico y considerado el padre de la Ciencia de Datos se cuestiona el futuro de la estadística como ciencia empírica. A principios de los 70 Peter Drucker, consultor de negocio, introduce el término 'Trabajador del conocimiento' y en paralelo Peter Naur acuño el término que utilizamos actualmente como Data Science (Ciencia de Datos).
Ya para los 80 el uso de Microsoft Excel se generaliza como la herramienta base de los Trabajadores del conocimiento y a finales de los 90 internet conecta el mundo y pone a disposición de todos el acceso al conocimiento. Este primer internet también se conoce como internet 1.0 o del contenido. Seguidamente, en los 2000, surge el boom de las DOTCom. Todas las empresas ponen su negocio online. Se empieza a gestar el internet 2.0 o de los servicios (tiendas online, acceso a plataformas web, contratación de servicios). Por aquel entonces -principios de los 2000- William S. Cleveland, informático y estadístico, introduce la Ciencia de Datos como una disciplina unificada e independiente.
Desde el 2005 en adelante las empresas de servicios comienzan a transformar la experiencia de usuario basada en datos y analítica. Las redes sociales toman peso y surge lo que conocemos como el internet 3.0 o de las personas donde lo que prima es estar conectados. El uso extensivo y generalizado de los smartphones da lugar al comienzo de la era de economía bajo demanda, surgen empresas como Uber, Airbnb, Amazon, Netflix (vertical de streaming) entre otras. Los dispositivos que utilizamos en el día a día como electrodomésticos y coches están cada vez más interconectados en lo que se ha denominado el internet 4.0 o de las cosas (IoT o Internet of Things).
La previsión es que se extienda el uso de Machine Learning e Inteligencia Artificial a cada vez más ámbitos gracias a la tecnificación de usuarios y especialistas; y la generalización, aumento y abaratamiento de la capacidad de cálculo.
¿Qué hay de verdad en la Ciencia de Datos? Este es uno de los casos donde la realidad supera a la ficción. La Ciencia de Datos tiene una particularidad que explica su crecimiento y es que es una disciplina muy accesible, pues el conocimiento y la tecnología están disponibles de forma generalizada y a muy bajo coste.
Hace unos meses comentaba Enrique Tellado en una mesa redonda algo, a priori, utópico: la posibilidad de vivir en una sociedad donde, gracias a la tecnología, el trabajo tal y como lo entendemos en jornadas de 8 horas diarias, no fuera necesario. Dejando a un lado los matices políticos, esta es una utopía que podríamos ver materializada a finales de este siglo. Esto por no hablar por los avances en el mundo médico y en otras disciplinas. Como defiende Steven Pinker en su obra 'Los Ángeles que Llevamos Dentro', el progreso como sociedad es innegable, cada vez vivimos mejor. En esto la tecnología, la ciencia de datos y la inteligencia artificial están jugando un papel clave.

"""

In [27]:
# Choose how to split the text: by paragraphs or by N words
chunks = split_into_paragraphs(text_input)  # Or use split_into_n_words(text_input, N)
#chunks = split_into_n_words(text_input, 50)
# Generate embeddings for each chunk
embeddings = [get_embedding(chunk) for chunk in chunks]

In [28]:
# Retrieve the most relevant chunks based on a question
question = "Que es la ciencia de datos?"
relevant_chunks = get_relevant_chunks(question, embeddings, chunks)
print(relevant_chunks[0])  # This will give you the most relevant chunk.
#print(relevant_chunks[1])
#print(relevant_chunks[2])

¿Qué hay de verdad en la Ciencia de Datos? Este es uno de los casos donde la realidad supera a la ficción. La Ciencia de Datos tiene una particularidad que explica su crecimiento y es que es una disciplina muy accesible, pues el conocimiento y la tecnología están disponibles de forma generalizada y a muy bajo coste.


# Punto 4

4. What do you think that could make these types of systems more robust in terms of semantics and functionality?


Improving the robustness of question-answering systems in terms of semantics and functionality is an active area of research. Here are several strategies that could make these systems more robust:

1. **Better Pre-training on Diverse Data**:
   - Models like BERT, GPT, and their variants are pre-trained on large datasets. However, training them on even more diverse datasets, including data from various domains, languages, and cultures, can help them understand semantics better.
  
2. **Incorporate External Knowledge Bases**:
   - Integrating information from structured knowledge bases like Wikipedia, DBpedia, or the Semantic Web can enhance the system's understanding. This integration helps in grounding the textual data with real-world facts and entities.
   
3. **Fine-tuning on Domain-specific Data**:
   - For a specific application or domain, the model can be fine-tuned on relevant datasets to ensure it captures the domain's nuances and semantics.

4. **Improving Contextual Understanding**:
   - Using models that can consider broader contexts or longer sequences of text can be beneficial. For instance, models like Transformer-XL and Longformer are designed to handle longer contexts.

5. **Multimodal Models**:
   - Incorporating other forms of data, like images or sounds, can provide additional context. Models like CLIP and ViLBERT, which consider both text and images, can offer richer semantic understanding.

6. **Attention Visualization and Analysis**:
   - Visualizing where the model is "looking" in the input (using attention weights) can provide insights into its decision-making process, helping to identify areas of improvement.

7. **Regularization and Adversarial Training**:
   - Introducing noise or adversarial examples during training can make the model more robust to slight alterations in the input, ensuring consistent responses.

8. **Feedback Loops**:
   - Systems that can learn continuously from user feedback or corrections will be better aligned with human expectations over time.

9. **Better Handling of Ambiguity**:
   - Instead of providing a single answer, the model can be designed to return a list of possible answers or ask clarifying questions when faced with ambiguous queries.

10. **Ethical and Bias Considerations**:
   - Regularly auditing and fine-tuning the models to reduce biases will make them more trustworthy and semantically accurate. Using tools and datasets explicitly designed for bias detection and mitigation can be beneficial.

11. **Incorporate Symbolic Reasoning**:
   - Hybrid models that combine deep learning with symbolic reasoning can perform logical deductions, ensuring that the system doesn't just rely on pattern recognition but also on logical rules.

12. **Continual Learning**:
   - Models that can learn over time without forgetting previously learned information can adapt to evolving languages and semantics.

13. **Multilingual and Cross-lingual Models**:
   - These models can transfer knowledge between languages, helping in understanding semantics in a more universal context.

By combining these strategies, one can build a question-answering system that is not only functionally robust but also semantically accurate and aligned with human expectations.